# Investigating bilby

Trying to learn what goes into bilby etc.

## Preamble

In [10]:
import psutil
p = psutil.Process()
p.cpu_affinity([0])

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from tqdm import tqdm
import json
import numpy as np

import jax
print(jax.devices())
import jax.numpy as jnp
from jimgw.prior import Uniform, Composite
import matplotlib.pyplot as plt
import pickle

# from save_likelihoods import NAMING as naming 
# from save_likelihoods import PRIOR

t_c = -0.023603896693900103
trigger_time = 1187008882.43

[CpuDevice(id=0)]


## Loading data

In [2]:
bilby_path = "/home/twouters2/data/update_theta_jn_injection_14_result.json" # on snellius

In [3]:
# Load the bilby data 
print("Loading bilby data")
with open(bilby_path, "r") as f:
    bilby_data = json.load(f)
bilby_posterior = bilby_data["posterior"]["content"]

bilby_naming = ['chirp_mass',
          'mass_ratio',
          'chi_1',
          'chi_2',
          'lambda_1',
          'lambda_2',
          'luminosity_distance',
          'geocent_time',
          'phase',
          'theta_jn',
          'psi',
          'ra',
          'dec'] # note we also have: # 'log_likelihood', 'log_prior'

bilby_chains = np.array([bilby_posterior[name] for name in bilby_naming])
bilby_chains = bilby_chains.T

print(np.shape(bilby_chains))

Loading bilby data
(12189, 13)


## Check bilby setup

In [5]:
print(bilby_data.keys())

dict_keys(['label', 'outdir', 'sampler', 'log_evidence', 'log_evidence_err', 'log_noise_evidence', 'log_bayes_factor', 'priors', 'posterior', 'injection_parameters', 'meta_data', 'search_parameter_keys', 'fixed_parameter_keys', 'constraint_parameter_keys', 'sampling_time', 'sampler_kwargs', 'use_ratio', 'information_gain', 'log_likelihood_evaluations', 'log_prior_evaluations', 'num_likelihood_evaluations', 'samples', 'nested_samples', 'parameter_labels', 'parameter_labels_with_unit', 'version'])


In [6]:
priors = bilby_data["priors"]
injection_parameters = bilby_data["injection_parameters"]

In [8]:
for key, item in injection_parameters.items():
    print(key, item)

chirp_mass 1.9568677619484602
mass_ratio 0.8774162003394455
chi_1 0.009735110398633173
chi_2 0.048980590878307986
luminosity_distance 126.18218167071768
theta_jn 2.640861949215473
psi 0.5161579022094159
phase 3.238585218011449
geocent_time 1187008882.43
ra 5.978101120559757
dec -0.01944744929956209
lambda_1 1064.683599128085
lambda_2 3723.7220879878337


In [9]:
for key, item in priors.items():
    print(key, item)

mass_1 {'__prior__': True, '__module__': 'bilby.core.prior.base', '__name__': 'Constraint', 'kwargs': {'minimum': 0.5, 'maximum': 5, 'name': 'mass_1', 'latex_label': '$m_1$', 'unit': None}}
mass_2 {'__prior__': True, '__module__': 'bilby.core.prior.base', '__name__': 'Constraint', 'kwargs': {'minimum': 0.5, 'maximum': 5, 'name': 'mass_2', 'latex_label': '$m_2$', 'unit': None}}
mass_ratio {'__prior__': True, '__module__': 'bilby.core.prior.analytical', '__name__': 'Uniform', 'kwargs': {'minimum': 0.125, 'maximum': 1.0, 'name': 'mass_ratio', 'latex_label': '$q$', 'unit': None, 'boundary': None}}
chirp_mass {'__prior__': True, '__module__': 'bilby.core.prior.analytical', '__name__': 'Uniform', 'kwargs': {'minimum': 1.9468677619484602, 'maximum': 1.9668677619484602, 'name': 'chirp_mass', 'latex_label': '$\\mathcal{M}$', 'unit': '$M_{\\odot}$', 'boundary': None}}
luminosity_distance {'__prior__': True, '__module__': 'bilby.gw.prior', '__name__': 'UniformComovingVolume', 'kwargs': {'minimum'

In [12]:
min_geocent_time = 1187008882.3300002
max_geocent_time = 1187008882.53

range = max_geocent_time - min_geocent_time
middle = (max_geocent_time + min_geocent_time) / 2

diff_trigger_time = trigger_time - middle
print("diff_trigger_time", diff_trigger_time)

diff_trigger_time 0.0
